In [1]:
cd '/home/user/project/python/make_local/'

/home/user/project/python/make_local


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from importlib import reload
import sys
import rms
import jax 
import numpy as np
import jax.numpy as jnp
from jax.config import config
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import nsp
import numpy as np
reload(rms)
groundstate = np.load("../exact/test/out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")


/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
U = np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/u/0.npy")
H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]

In [5]:
ur = rms.unitary.UnitaryRiemanGenerator(8, jax.random.PRNGKey(0), np.float64)

UnitaryRiemanGenerator is initialized


In [6]:
x = groundstate.reshape([8]*4)
x = x.transpose([0,2,1,3]).reshape(-1)
x = jnp.array(x)
# loss = rms.loss.QuasiEnergy(jnp.array(H0), 8, x, np.float64)
mesLoss_list = [
    rms.loss.MinimumEnergy(jnp.array(h), 8, np.float64) for h in H_list
    ]
mesLoss = rms.loss.MeanMultiLoss(mesLoss_list)

upper bounds : 3.079512
target       : 3.000000


In [7]:
t = rms.functions.stoquastic(jnp.kron(U, U).T @ H_list[0] @ jnp.kron(U, U))
jnp.linalg.eigvalsh(t)

Array([-1.00034916e+00, -9.79097151e-01, -9.54217542e-01, -9.22754087e-01,
       -8.47858563e-01, -8.27327814e-01, -8.03594034e-01, -7.74211543e-01,
       -7.50342166e-01, -7.19016830e-01, -6.94152126e-01, -6.72754991e-01,
       -5.97851422e-01, -5.68802974e-01, -5.45086775e-01, -5.24212930e-01,
       -1.86787936e-04,  2.28647186e-04,  8.12965998e-03,  1.61914494e-02,
        2.39737064e-02,  2.46443726e-02,  5.26298037e-02,  8.32114952e-02,
        8.34328182e-02,  8.34552292e-02,  9.81950029e-02,  9.86915642e-02,
        9.99084906e-02,  1.07549837e-01,  1.19775868e-01,  1.19967414e-01,
        1.66614088e-01,  1.66761855e-01,  2.49801149e-01,  2.49835676e-01,
        2.49900315e-01,  2.49999561e-01,  2.49999675e-01,  2.50044942e-01,
        2.50226277e-01,  2.73973989e-01,  2.84817010e-01,  2.88121759e-01,
        2.98172437e-01,  3.15221184e-01,  3.15351509e-01,  3.33211884e-01,
        3.33432977e-01,  3.33453582e-01,  3.48198797e-01,  3.49104132e-01,
        3.51217641e-01,  

In [10]:
u = ur.reset_matrix()
# u = jnp.array(U)
opt_init, opt_update, get_unitary = rms.optimizer.momentum(0.01, 0.3)
opt_state = opt_init(u)

In [12]:
def step(step, opt_state):
    value, grads = jax.value_and_grad(mesLoss)(get_unitary(opt_state))
    riemannian_grad = ur.riemannian_grad(u, grads)
    opt_state = opt_update(step, riemannian_grad, opt_state)
    return value, opt_state
for i in range(10000):
    value, opt_state = step(i, opt_state)
    u = get_unitary(opt_state)
    # print(jnp.linalg.norm(u @ u.T - jnp.eye(8)))
    print(value)

PyTreeDef(*)
3.793801872578438
PyTreeDef(*)
3.793540885527282
PyTreeDef(*)
3.7960833723174003
PyTreeDef(*)
3.7880898744929823
PyTreeDef(*)
3.798076359576701
PyTreeDef(*)
3.779108590043689
PyTreeDef(*)
3.7937821341728073
PyTreeDef(*)
3.7830308861353856
PyTreeDef(*)
3.785408368973421
PyTreeDef(*)
3.7759930616374517
PyTreeDef(*)
3.7933465432387736
PyTreeDef(*)
3.7741735632324627
PyTreeDef(*)
3.7872150583572584
PyTreeDef(*)
3.7717171261065126
PyTreeDef(*)
3.78292870420564
PyTreeDef(*)
3.771304962279557
PyTreeDef(*)
3.7760983081103223
PyTreeDef(*)
3.761649624941146
PyTreeDef(*)
3.7841510690378417
PyTreeDef(*)
3.7502822774955753
PyTreeDef(*)
3.7948216062462263
PyTreeDef(*)
3.7503520853077603
PyTreeDef(*)
3.7662974555494726
PyTreeDef(*)
3.7586291305881376
PyTreeDef(*)
3.7700979405678066
PyTreeDef(*)
3.7602334025027773
PyTreeDef(*)
3.765113911561391
PyTreeDef(*)
3.754174161259166
PyTreeDef(*)
3.7663730276646317
PyTreeDef(*)
3.7520231514883617
PyTreeDef(*)
3.7571723834010333
PyTreeDef(*)
3.7501

KeyboardInterrupt: 

In [63]:
get_unitary(opt_state)

Array([[ 0.02637667, -0.62523228, -0.25824682,  0.07652412, -0.27652208,
        -0.38720954, -0.00610629, -0.55619447],
       [-0.51539503,  0.33571185,  0.35349429,  0.27117806,  0.10260862,
        -0.03989071,  0.31229313, -0.55838893],
       [-0.04371901, -0.0180621 ,  0.28708313,  0.24463985, -0.86876533,
         0.18117465,  0.11503734,  0.22398933],
       [-0.03605591, -0.36168213,  0.64388014, -0.32749439,  0.15367543,
        -0.41718214,  0.27541101,  0.2740728 ],
       [ 0.01582695, -0.34372539,  0.36680637, -0.2736082 ,  0.06825381,
         0.68720487, -0.29529531, -0.32858958],
       [-0.79769207, -0.24126144, -0.13116612,  0.07727826,  0.03992199,
        -0.04169526, -0.42303294,  0.315108  ],
       [-0.29819277, -0.07747142, -0.39826263, -0.50908523, -0.10746989,
         0.2787619 ,  0.62583567,  0.04138408],
       [ 0.07631887, -0.42299142, -0.04946863,  0.6438758 ,  0.33804042,
         0.29962748,  0.38757305,  0.20948032]], dtype=float64)

In [40]:
ur.reset_params()
val = mesLoss(ur.matrix)
# print(val)
grad = jax.grad(mesLoss)(ur.matrix)
riemannian_grad = ur.riemannian_grad(grad)
print(riemannian_grad)



[[ 0.          0.04848574 -0.14949599 -0.12895075  0.08539086  0.21497774
  -0.30938472  0.146692  ]
 [-0.04848574  0.         -0.13104768  0.1381185   0.15857814  0.02893414
   0.03815241  0.12007688]
 [ 0.14949599  0.13104768  0.         -0.1519691   0.01669821  0.09771466
  -0.07352611  0.08505269]
 [ 0.12895075 -0.1381185   0.1519691   0.          0.14733817  0.00988164
   0.03453659  0.04185871]
 [-0.08539086 -0.15857814 -0.01669821 -0.14733817  0.          0.19016541
  -0.0569112   0.27680873]
 [-0.21497774 -0.02893414 -0.09771466 -0.00988164 -0.19016541  0.
  -0.06713462 -0.1142042 ]
 [ 0.30938472 -0.03815241  0.07352611 -0.03453659  0.0569112   0.06713462
   0.          0.17117901]
 [-0.146692   -0.12007688 -0.08505269 -0.04185871 -0.27680873  0.1142042
  -0.17117901  0.        ]]


In [15]:
jnp.linalg.norm(riemannian_grad)

Array(1.03253213e-15, dtype=float64)

In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
import jax.numpy as jnp
import jax
# jax.config.update("jax_platform_name", args.platform)
A = jax.random.normal(jax.random.PRNGKey(0), (3, 3))
# A = jax.random.normal(jax.random.PRNGKey(0), (4096, 4096))
# A = A + A.T

In [10]:
4096 ** 2 * 8

134217728

In [7]:
E, V = jnp.linalg.eigh(A)

In [ ]:
def f(x):
    return jnp.linalg.norm(jnp.linalg.eigvalsh(x)[0])

## Gradient